In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import multiprocessing
import time
import tensorflow as tf
drive.mount('/content/drive')
import tensorflow as tf
import os.path
tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/device:GPU:0'

In [0]:
import glob
glob.glob("/content/drive/My Drive/google_notebook/M5_traindata2/*")

['/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sample_submission.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a']

In [0]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 4 #スレッド数
process_num = 8 #プロセス数

print(len(train_data))

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num):
    for ii in range(start_n, len(train_data)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a["value"] = tra_sir[6:].to_list()

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

thread_list = []
with tf.device('/cpu:0'):
  for i in range(thread_num):
      thread = threading.Thread(target=traindata2_making, args=(i,))
      thread.start()
      thread_list.append(thread)
      print(i)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

with tf.device('/gpu:0'):
  for i2 in range(thread_num, thread_num + process_num):
      mp = threading.Thread(target=traindata2_making, args=(i2,))
      mp.start()
      thread_list.append(thread)
      print(i2)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

while(True):
  print('\r', end='')
  print(current_num, end='')
  sleep_seconds = 2
  time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

30490
0
1
2
3


KeyboardInterrupt: ignored

In [0]:
ii = 1413
if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
  train_data2a = train_data2a0.copy()
  tra_sir = train_data.iloc[ii]
  for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
  train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
  train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
  train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
  train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

  sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
  sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
  sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

  train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

  train_data2a["value"] = tra_sir[6:].to_list()

  train_data2a.to_csv(
      "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
else: print("files exist")

files exist


In [0]:
df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(1413) + ".csv")

In [0]:
os.remove("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(1413) + ".csv")

In [0]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_0.csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt", mode='w') as f:
    f.write(str(1))

In [0]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_all.csv", index=False)

1400

In [0]:
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt") as f:
    ii = int(f.read()) + 1
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt", mode='w') as f:
    f.write(str(ii))

In [0]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_all.csv", index=False)

22486

In [0]:
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt") as f:
    ii = int(f.read()) + 1
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt", mode='w') as f:
    f.write(str(ii))

In [0]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_all.csv", index=False)